In [ ]:
# outputttt3_chunk1.diar

#open all the files like this

import sys
import glob
import pickle

import numpy as np
#it has to have chunkx.diar

files = glob.glob("out/*_chunk*.diar")
print(files)

#sort by chunk number
files.sort(key=lambda x: int(x.split("_chunk")[1].split(".diar")[0]))

print(files)


diars = []

for file in files:
    #read pickle
    with open(file, 'rb') as f:
        diar = pickle.load(f)
        diars.append(diar)




diars = np.array(diars)
print(diars.shape)

fig = go.Figure()


t=0
for chunk in diars:
    print(chunk.shape)

    for i in range(6):
        #shift x by t
        fig.add_trace(go.Scatter( x=np.arange(t, t+chunk.shape[1]), y=chunk[i,:]*0.5 + (i), mode='lines', name=f"Speaker {i}"))
    t+=chunk.shape[1]

fig.show()

#(30, 6, 1000)
#30 chunks, 6 speakers, 1000 frames

#merge the chunks to create 30*1000 frames

# diars = diars.reshape(-1, 6)
# #swap the axes
# diars = diars.T
# # print(diars.shape)
# # diars=diars[4]
# print(diars.shape)
# #plot using plotly
# import plotly.graph_objects as go

# fig = go.Figure()

# for i in range(6):
#     #each speaker at own y 
#     fig.add_trace(go.Scatter( y=diars[i,:]*0.5 + (i), mode='lines', name=f"Speaker {i}"))


# fig.show()

In [ ]:
len(diars[0])

In [ ]:
import glob

# Retrieve all WAV files matching the pattern
wavs = glob.glob("out/*_chunk*_mic*.wav")

# Sort the files by chunk number first, then by mic number
wavs.sort(key=lambda x: (int(x.split("_chunk")[1].split("_mic")[0]), int(x.split("_mic")[1].split(".wav")[0])))

print(wavs)




In [ ]:

# Retrieve all WAV files for mic 0
import soundfile as sf
from IPython.display import Audio
import numpy as np
import plotly.graph_objects as go
import glob

#load all wavs again
wavs = []
#outputttt3_mic0_.wav
for f in sorted(glob.glob('./out/*.wav'), key=lambda x: int(x.split('_mic')[1].split('.wav')[0].split('_')[0])):
    print(f)
    w, sr = sf.read(f)
    wavs.append(w)


num_sources = wavs[0].shape[1]

print('Number of sources:', num_sources)


#for each surce find the wav with the most energy and keep it

best = [[] for _ in range(len(wavs))]
chunk_size = 10 * sr

#go through all the chunks
for i in range(0, len(wavs[0]), chunk_size):
    #get the chunk
    chunk = np.array([w[i:i+chunk_size] for w in wavs])

    print(chunk.shape)
    #(8, 160000, 6) 

    #loop through all microphones
    for j,mic in enumerate(chunk):

        print(mic.shape) #(160000, 6)

        #calculate the energy for each source
        energy = np.sum(mic**2, axis=0)

        print(energy.shape)

        #find the source with the most energy
        best_source = np.argmax(energy)

        print(i)
        #extend the best sources
        best[j] = np.concatenate((best[j], mic[:, best_source])) if len(best) > 0 else mic[:, best_source]


best = np.array(best)
print(best.shape)




            

                


      
sf.write('./best_sources.wav', np.array(best).T, sr)


#plot the best sources
fig = go.Figure()
mic = 1
seconds = 20
#plot mic 0 all channels
for i in range(num_sources):
    fig.add_trace(go.Scatter (y=wavs[mic][:seconds*sr, i]*0.5 + i, mode='lines', name=f"Speaker {i}"))


#show diar

fig.show()


    


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Create synthetic signals
t = np.linspace(0, 10, 1200)  # Time vector
signal1 = np.sin(2 * np.pi * t)  # Sine wave
signal2 = np.cos(2 * np.pi * t)  # Cosine wave

# Combine signals to create overlapping segments
combined_signal = signal1 + 0.5 * signal2

# Plot the synthetic signals
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=signal1, mode='lines', name='Signal 1'))
fig.add_trace(go.Scatter(x=t, y=signal2, mode='lines', name='Signal 2'))
fig.add_trace(go.Scatter(x=t, y=combined_signal, mode='lines', name='Combined Signal'))

fig.update_layout(title='Synthetic Signals', xaxis_title='Time', yaxis_title='Amplitude')
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Step 1: Generate Synthetic Signals
t = np.linspace(0, 10, 1000)  # Time vector
signal1 = np.sin(2 * np.pi * t)  # Sine wave
signal2 = np.cos(2 * np.pi * t)  # Cosine wave
combined_signal = signal1 + 0.5 * signal2

# Plot the synthetic signals
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=signal1, mode='lines', name='Signal 1'))
fig.add_trace(go.Scatter(x=t, y=signal2, mode='lines', name='Signal 2'))
fig.add_trace(go.Scatter(x=t, y=combined_signal, mode='lines', name='Combined Signal'))
fig.update_layout(title='Synthetic Signals', xaxis_title='Time', yaxis_title='Amplitude')
fig.show()

# Step 2: Chunking with Overlap
chunk_len = 200
overlap = 100

chunks = []
start = 0
while start < len(combined_signal):
    end = start + chunk_len
    if end > len(combined_signal):
        end = len(combined_signal)
    chunk = combined_signal[start:end]
    
    # Pad the chunk if it's shorter than the expected chunk length
    if len(chunk) < chunk_len:
        chunk = np.pad(chunk, (0, chunk_len - len(chunk)), 'constant')
        
    chunks.append(chunk)
    start += overlap

chunks = np.array(chunks)

# Plot the chunks
for i, chunk in enumerate(chunks):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(len(chunk)), y=chunk, mode='lines', name=f'Chunk {i+1}'))
    fig.update_layout(title=f'Chunk {i+1}', xaxis_title='Samples', yaxis_title='Amplitude')
    fig.show()

# Step 3: Permutation and Stitching Visualization
def visualize_stitching(chunks, permutations):
    stitched_signal = np.zeros(len(chunks) * chunk_len - (len(chunks) - 1) * overlap)
    start = 0
    for i, perm in enumerate(permutations):
        chunk = chunks[i]
        end = start + chunk_len
        if start == 0:
            stitched_signal[start:end] = chunk
        else:
            stitched_signal[start:start + overlap] = (stitched_signal[start:start + overlap] + chunk[:overlap]) / 2
            stitched_signal[start + overlap:end] = chunk[overlap:]
        start += chunk_len - overlap
    
    return stitched_signal

# Define permutations (identity permutation for simplicity)
permutations = [np.arange(chunk_len) for _ in range(len(chunks))]

# Visualize the stitched signal
stitched_signal = visualize_stitching(chunks, permutations)

# Plot the stitched signal
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(stitched_signal)), y=stitched_signal, mode='lines', name='Stitched Signal'))
fig.update_layout(title='Stitched Signal', xaxis_title='Samples', yaxis_title='Amplitude')
fig.show()